## LAB 2 ML 2023-24. - OPTIONAL TASKS FOR COURSE PROJECT (1 POINT).

FILL UP THIS BOX WITH YOUR DETAILS

**NAME AND NIP**:
Anyiel Fernandes Araujo 779374
Pablo Garcia Garcia 781020

### This task is built on an oficial Detectron2 Tutorial for object detection, to run inference on images or videos, with an existing detectron2 model

### Detectron2 Beginner's Tutorial
https://detectron2.readthedocs.io/en/latest/tutorials/getting_started.html

© Copyright 2019-2020, detectron2 contributors Revision 1ad5759b.


### Different from the other notebooks in this Lab, this one uses pytorch framework, so you can see an example with this other framework.


# Install detectron2

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Run a pre-trained detectron2 model

We first download an image from the COCO dataset:

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O input.jpg
im = cv2.imread("./input.jpg")
cv2_imshow(im)

Then, we create a detectron2 config and a detectron2 `DefaultPredictor` to run inference on this image.

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [ ]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
print(outputs["instances"].pred_classes)
print(outputs["instances"].pred_boxes)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

### QUESTION 1:
Change two parameters in this CFG and explain what you have changed and what you expect to change in the output, or what you see is different analyzing the new predictions.

For example, find the parameter that allows more detections although less confident ones, and change the model used for the predictions (You can find plenty of models from detectron2's model zoo https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md)

In [ ]:
# With this print you can take a look to all the configuration parameters set for this particular inference task
# if you want to explore something else than the "basic" changes suggested above (threshold and model used)
print(cfg)

In [ ]:
# put here your snippet of code to run inference with your modification 1

In [ ]:
# put here your snippet of code to run inference with your modification 2

ANSWER 1: [YOUR ANSWER HERE] (max 5 lines)

### QUESTION 2:
After running the model on your own *image test set* (10 images that you take with your phone. 5 that you thought are *easy* for this model. 5 that you thought would be difficult for the model).

Evaluate **qualitatively (no need to label your images)** and **discuss results**. What works well or not? Why? What works or doesn’t as you expected?

NOTE: careful if your image has different size than the example, no need to change anything in CFG only re-launch all the code (visualizer included) to take the right dimensions in the display

In [ ]:
# put here your snippet of code to run the inference on your test images and display the results

ANSWER 2: [YOUR ANSWER HERE] (max 10 lines)

# Other types of builtin models in Detectron2

If you are curious, detectron2 provides a very powerful platform, with additional models other than simple detection, these are some other examples with existing models

In [ ]:
# Inference with a keypoint detection model
cfg = get_cfg()   # get a fresh new config
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
outputs = predictor(im)
v = Visualizer(im[:,:,::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
# Inference with a panoptic segmentation model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)
panoptic_seg, segments_info = predictor(im)["panoptic_seg"]
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
cv2_imshow(out.get_image()[:, :, ::-1])